In [2]:
import requests
import json
import pandas as pd
from getpass import getpass
from pymongo import MongoClient
import folium
import os

from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

C:\Users\ateso\AppData\Local\Temp\ipykernel_22876\2666485127.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [19]:
token = getpass()

········


In [136]:
def places (venue, lat, lon):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&limit=50"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    response = requests.get(url, headers=headers).json()
    return response

result = places ("Oficinas", "41.3977461", "2.1876744")

json_file_path = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Offices.json"

with open(json_file_path, "w") as json_file:
    json.dump(result, json_file, indent=2)

print(f"Result has been saved to {json_file_path}")


Result has been saved to C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Offices.json


In [125]:
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [48]:
def to_csv(database_name, collection_name):
   
    client = MongoClient("localhost:27017")
    db = client[database_name]
    c = db[collection_name]

    filter = {
        "results.name": {"$exists": True},
        "results.geocodes.main": {"$ne": None}
    }

    projection = {
        "results.name": 1,
        "results.geocodes.main": 1
    }

    sorting = [("results.0.name", 1)]

    result_cursor = c.find(filter, projection).sort(sorting)
    result_list = list(result_cursor)

   
    names_and_coordinates = [
        (item['name'], item['geocodes']['main']['latitude'], item['geocodes']['main']['longitude'])
        for item in result_list[0]['results']
    ]

  
    df = pd.DataFrame(names_and_coordinates, columns=['Name', 'Latitude', 'Longitude'])

  
    df.to_csv("C:\\Users\\ateso\\Desktop\\Ironhack\\Project-3\\Data\\acsv\\new_csv.csv", index=False)


In [36]:
def create_gym_map(csv_file_path):
  
    df_gym = pd.read_csv(csv_file_path)

 
    map_center = [df_gym['Latitude'].mean(), df_gym['Longitude'].mean()]

    
    gym_map = folium.Map(location=map_center, zoom_start=13)


    for index, row in df_gym.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='red', icon='dumbbell', prefix='fa')
        ).add_to(gym_map)

    return gym_map

In [39]:
def create_bars_map (csv_file_path):
    
    df_bars = pd.read_csv(csv_file_path)

    map_center = [df_bars['Latitude'].mean(), df_bars['Longitude'].mean()]

    bars_map = folium.Map(location=map_center, zoom_start=13)

    for index, row in df_bars.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='blue', icon='beer', prefix='fa')
        ).add_to(bars_map)


    return bars_map

In [40]:

def create_restaurants_map (csv_file_path):
    
    df_restaurants = pd.read_csv(csv_file_path)

    map_center = [df_restaurants['Latitude'].mean(), df_restaurants['Longitude'].mean()]

    restaurants_map = folium.Map(location=map_center, zoom_start=13)

    for index, row in df_restaurants.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='lightblue', icon='utensils', prefix='fa')
        ).add_to(restaurants_map)


    return restaurants_map

In [42]:

def create_schools_map (csv_file_path):
    
    df_schools = pd.read_csv(csv_file_path)

    map_center = [df_schools['Latitude'].mean(), df_schools['Longitude'].mean()]

    schools_map = folium.Map(location=map_center, zoom_start=16)

    for index, row in df_schools.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='lightred', icon='school', prefix='fa')
        ).add_to(schools_map)


    return schools_map

In [43]:

def create_others_map (csv_file_path):
    
    df_others = pd.read_csv(csv_file_path)

    map_center = [df_others['Latitude'].mean(), df_others['Longitude'].mean()]

    others_map = folium.Map(location=map_center, zoom_start=16)

    for index, row in df_others.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='orange', icon='building-columns', prefix='fa')
        ).add_to(others_map)


    return others_map

In [100]:
file_path = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Athens\Athens_all_data\Athens_data.csv"
dfAthens = pd.read_csv(file_path)
dfAthens.sample(8)

,Name,Latitude,Longitude,Local_type
64,Pitbull Gym,37.955239,23.711949,Gym
41,Qassabah,37.976423,23.776368,Restaurants
131,Ευαγγελική Σχολή,37.931476,23.713284,Schools
288,Μέντα,38.070985,23.812476,Bar
206,K1 & K2 Oaka,38.041767,23.787371,Others
86,Yava Fitness Center - Burnazi,38.014755,23.707762,Gym
24,Naif,37.957171,23.719400,Restaurants
174,Esplanada (Εσπλανάδα),37.936128,23.691079,Others


In [105]:
Athens_lat = 37.9871384
Athens_lon = 23.7183808

Athens_map = folium.Map(location=[Athens_lat, Athens_lon], zoom_start=15)


bar_group = folium.FeatureGroup(name='Bars')
gym_group = folium.FeatureGroup(name='Gyms')
school_group = folium.FeatureGroup(name='Schools')
restaurant_group = folium.FeatureGroup(name='Restaurants')
others_group = folium.FeatureGroup(name='Others')

for index, row in dfAthens.iterrows():
    Local_type = row["Local_type"]
    Name = row["Name"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    if Local_type == "Bar":
        icon = folium.Icon(color='blue', icon_color='black', icon='beer', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(bar_group)
    
    elif Local_type == "Gym":
        icon = folium.Icon(color='red', icon_color='white', icon='dumbbell', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(gym_group)
    
    elif Local_type == "Schools":
        icon = folium.Icon(color='lightred', icon_color='white', icon='school', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(school_group)
    
    elif Local_type == "Restaurants":
        icon = folium.Icon(color='lightblue', icon_color='white', icon='utensils', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(restaurant_group)
    
    elif Local_type == "Others":
        icon = folium.Icon(color='orange', icon_color='white', icon='building-columns', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(others_group)
    
    else:
        print(f"Unknown Local_type: {Local_type}")
        continue


bar_group.add_to(Athens_map)
gym_group.add_to(Athens_map)
school_group.add_to(Athens_map)
restaurant_group.add_to(Athens_map)
others_group.add_to(Athens_map)


folium.LayerControl().add_to(Athens_map)

mad_map


In [104]:
file_path = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Barcelona\Barcelona_all_data_300\Barcelona_data.csv"
dfBarcelona = pd.read_csv(file_path)
dfBarcelona.sample(8)

,Name,Latitude,Longitude,Local_type
83,Gimnasio McFIT Barcelona Universitat,41.388035,2.165913,Gym
71,Centre Esportiu Joan Miró,41.378479,2.150979,Gym
0,Can Dendê,41.398360,2.198345,Restaurants
167,Jardins del Mirador,41.367574,2.169399,Others
95,Activitats Esportives Eixample,41.380382,2.149910,Gym
12,Ciutat Comtal,41.388858,2.166857,Restaurants
210,Casa Lolea Barcelona,41.388833,2.177238,Bar
92,Duet Fit Sagrada Familia,41.407336,2.180823,Gym


In [107]:
Barcelona_lat = 41.3977461
Barcelona_lon = 2.1876744

Barcelona_map = folium.Map(location=[Barcelona_lat, Barcelona_lon], zoom_start=15)

bar_group = folium.FeatureGroup(name='Bars')
gym_group = folium.FeatureGroup(name='Gyms')
school_group = folium.FeatureGroup(name='Schools')
restaurant_group = folium.FeatureGroup(name='Restaurants')
others_group = folium.FeatureGroup(name='Others')

for index, row in dfAthens.iterrows():
    Local_type = row["Local_type"]
    Name = row["Name"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    if Local_type == "Bar":
        icon = folium.Icon(color='blue', icon_color='black', icon='beer', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(bar_group)
    
    elif Local_type == "Gym":
        icon = folium.Icon(color='red', icon_color='white', icon='dumbbell', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(gym_group)
    
    elif Local_type == "Schools":
        icon = folium.Icon(color='lightred', icon_color='white', icon='school', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(school_group)
    
    elif Local_type == "Restaurants":
        icon = folium.Icon(color='lightblue', icon_color='white', icon='utensils', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(restaurant_group)
   
    elif Local_type == "Others":
        icon = folium.Icon(color='orange', icon_color='white', icon='building-columns', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(others_group)
    
    else:
        print(f"Unknown Local_type: {Local_type}")
        continue


bar_group.add_to(Barcelona_map)
gym_group.add_to(Barcelona_map)
school_group.add_to(Barcelona_map)
restaurant_group.add_to(Barcelona_map)
others_group.add_to(Barcelona_map)


folium.LayerControl().add_to(Barcelona_map)

Barcelona_map


In [119]:
file_path = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Lisboa\Lisboa_all_300\Lisboa_data.csv"
dfLisboa = pd.read_csv(file_path)
dfLisboa['Local_type'] = dfLisboa['Local_type'].str.capitalize()

dfLisboa.sample(8)

,Name,Latitude,Longitude,Local_type
271,Tapa Bucho,38.714233,-9.145118,Bar
58,Fitness Hut,38.724601,-9.146344,Gym
147,Instituto Português de Oncologia de Francisco ...,38.739775,-9.161241,Schools
223,Pavilhão do Clube Atlético de Campo de Ourique,38.718702,-9.161957,Others
210,Campo Treinos SLB,38.753245,-9.182461,Others
36,Restaurante Mercado do Peixe,38.712818,-9.203415,Restaurants
139,Cais,38.768873,-9.380564,Schools
192,Parque Marechal Carmona,38.694177,-9.422922,Others


In [120]:
Lisboa_lat = 38.7425777
Lisboa_lon = -9.1436525

Lisboa_map = folium.Map(location=[Lisboa_lat, Lisboa_lon], zoom_start=15)


bar_group = folium.FeatureGroup(name='Bars')
gym_group = folium.FeatureGroup(name='Gyms')
school_group = folium.FeatureGroup(name='Schools')
restaurant_group = folium.FeatureGroup(name='Restaurants')
others_group = folium.FeatureGroup(name='Others')

for index, row in dfLisboa.iterrows():
    Local_type = row["Local_type"]
    Name = row["Name"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    if Local_type == "Bar":
        icon = folium.Icon(color='blue', icon_color='black', icon='beer', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(bar_group)
    
    elif Local_type == "Gym":
        icon = folium.Icon(color='red', icon_color='white', icon='dumbbell', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(gym_group)
    
    elif Local_type == "Schools":
        icon = folium.Icon(color='lightred', icon_color='white', icon='school', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(school_group)
    
    elif Local_type == "Restaurants":
        icon = folium.Icon(color='lightblue', icon_color='white', icon='utensils', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(restaurant_group)
   
    elif Local_type == "Others":
        icon = folium.Icon(color='orange', icon_color='white', icon='building-columns', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(others_group)
    
    else:
        print(f"Unknown Local_type: {Local_type}")
        continue


bar_group.add_to(Lisboa_map)
gym_group.add_to(Lisboa_map)
school_group.add_to(Lisboa_map)
restaurant_group.add_to(Lisboa_map)
others_group.add_to(Lisboa_map)


folium.LayerControl().add_to(Lisboa_map)

Lisboa_map

In [129]:
def generate_city_map(city, lat, lon, data_path):

    df = pd.read_csv(data_path)
    df['Local_type'] = df['Local_type'].str.capitalize()

    city_map = folium.Map(location=[lat, lon], zoom_start=15)

    bar_group = folium.FeatureGroup(name='Bars')
    gym_group = folium.FeatureGroup(name='Gyms')
    school_group = folium.FeatureGroup(name='Schools')
    restaurant_group = folium.FeatureGroup(name='Restaurants')
    others_group = folium.FeatureGroup(name='Others')

    for index, row in df.iterrows():
        Local_type = row["Local_type"]
        Name = row["Name"]
        latitude = row["Latitude"]
        longitude = row["Longitude"]

        if Local_type == "Bar":
            icon = folium.Icon(color='blue', icon_color='black', icon='beer', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(bar_group)

        elif Local_type == "Gym":
            icon = folium.Icon(color='red', icon_color='white', icon='dumbbell', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(gym_group)

        elif Local_type == "Schools":
            icon = folium.Icon(color='lightred', icon_color='white', icon='school', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(school_group)

        elif Local_type == "Restaurants":
            icon = folium.Icon(color='lightblue', icon_color='white', icon='utensils', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(restaurant_group)

        elif Local_type == "Others":
            icon = folium.Icon(color='orange', icon_color='white', icon='building-columns', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(others_group)

        else:
            print(f"Unknown Local_type: {Local_type}")
            continue

    bar_group.add_to(city_map)
    gym_group.add_to(city_map)
    school_group.add_to(city_map)
    restaurant_group.add_to(city_map)
    others_group.add_to(city_map)

   
    folium.LayerControl().add_to(city_map)

    return city_map

city_map = generate_city_map("Athens", 37.9871384, 23.7183808, r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Athens\Athens_all_data_300\Athens_data.csv")
city_map


In [184]:
generate_city_map("Paris", "48.836214", "2.340852", r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Paris\Paris_all_data_300\combined_paris_data.csv")

In [185]:
generate_city_map("Madrid", "40.4010201", "-3.7067751", r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Madrid\Madrid_all_data_300\combined_Madrid_data.csv")

In [131]:
create_heatmap(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Athens\Athens_all_data_300\Athens_data.csv")

In [133]:
create_heatmap(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Barcelona\Barcelona_all_data_300\Barcelona_data.csv")

In [134]:
create_heatmap(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Lisboa\Lisboa_all_300\Lisboa_data.csv")

In [144]:
create_heatmap(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Paris\Paris_all_data_300\combined_paris_data.csv")

In [187]:
create_heatmap(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Madrid\Madrid_all_data_300\combined_Madrid_data.csv")